In [1]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

/Users/tangotew/miniconda3/envs/llm_quant/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_or_download_model(model_name, model_dir):
    # Check if the model directory exists
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        # Download and save the model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        tokenizer.save_pretrained(model_dir)
        model.save_pretrained(model_dir)
    else:
        # Load the model and tokenizer from the local directory
        tokenizer = AutoTokenizer.from_pretrained(model_dir)
        model = AutoModelForCausalLM.from_pretrained(model_dir)
    
    return model, tokenizer

def generate_response(model, tokenizer, prompt, max_length=150):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [9]:
# Define the model name and directory
# Load the GPT-NeoX model and tokenizer
# Define the model name and directory
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model_dir = "../models/tiny-llama-1.1B"

# Load or download the model
model, tokenizer = load_or_download_model(model_name, model_dir)

In [10]:
# Example usage
prompt = "Write a Python function to fetch weather data from an API."
generated_code = generate_response(model, tokenizer, prompt)
print(f"Generated Code:\n{generated_code}")

Generated Code:
Write a Python function to fetch weather data from an API. The function should take in the API key as a parameter and return a dictionary containing the current temperature, humidity, and wind speed. The function should also handle errors and return an empty dictionary if the API call fails. The function should be well-documented and follow PEP 8 style guidelines.
